In [ ]:
#| default_exp core

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
"""Propose RAM-saving changes in a DataFrame"""

import pandas as pd
import numpy as np
from collections import namedtuple


# TODO
# more tests
#   test float64->float32->float16
# consider uint64/32/16/8
# does the "object" check work if col has non-str items?
# enable approx-equal with np.close (note for big nbrs, a big delta is "acceptable" with this)

# convert_dtypes converts e.g. int64 to Int64 (nullable) regardless of nulls, also obj->string
# so it doesn't save RAM but it does suggest new safer datatypes
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.convert_dtypes.html

# For a dtype count the nbr of conversions that aren't equal, the RAM cost
# of the conversion and the column name
AsType = namedtuple("AsType", ["dtype", "nbr_different", "nbytes", "col"])
_fields = (
    "column",
    "current_dtype",
    "proposed_dtype",
    "current_memory",
    "proposed_memory",
    "ram_usage_improvement",
)
Row = namedtuple("Row", _fields, defaults=(None,) * len(_fields))


def count_errors(ser: pd.Series, new_dtype):
    """After converting ser to new dtype, count whether items have isclose()"""
    tmp_ser = ser.astype(new_dtype)
    # metric will be a list of Trues if the change has equivalent value, False otherwise
    # checks for approx equal which may not be what we want
    # metric = np.isclose(ser, tmp_ser)

    ndiff = len(ser.compare(tmp_ser))  # pandas >= 1.1.0
    nbytes = tmp_ser.memory_usage(deep=True)
    as_type = AsType(new_dtype, ndiff, nbytes, ser.name)
    return as_type


def map_dtypes_to_choices(ser: pd.Series, optimize: str):
    if optimize == "memory":
        new_dtypes = {
            "int64": ["int32", "int16", "int8"],
            "float64": ["float32", "float16"],
            "object": ["category"],
        }
    elif optimize == "computation":
        new_dtypes = {
            "int64": ["int32", "int16", "int8"],
            "float64": ["float32", "float16"],
            "object": ["category"],
        }

    return new_dtypes.get(ser.dtype.name)

In [ ]:
#| export
def get_smallest_valid_conversion(ser: pd.Series, optimize: str):
    new_dtypes = map_dtypes_to_choices(ser, optimize)
    if new_dtypes:
        for new_dtype in reversed(new_dtypes):
            as_type = count_errors(ser, new_dtype)
            if as_type.nbr_different == 0:
                return as_type
    return None


def get_improvement(as_type: AsType, current_nbytes: int) -> pd.DataFrame:
    report = (None, None, None)
    ram_usage_improvement = current_nbytes - as_type.nbytes
    if ram_usage_improvement > 0:
        report = (
            as_type.nbytes,
            as_type.dtype,
            ram_usage_improvement,
        )
    return report

In [ ]:
#| export
def report_on_dataframe(
    df: pd.DataFrame, unit: str = "MB", optimize: str = "memory"
) -> pd.DataFrame:

    """[Report on columns that might be converted]
    Args:
        df ([type]): [description]
        unit (str, optional): [byte, MB, GB]. Defaults to "MB".
        optimize (str, optional): [memory, computation]. Defaults to memory.
        [memory]: The lowest memory dtype for float is fp16.
        [computation]: The lowest memory dtype for float is fp32.
    """

    unit_map = {"KB": 1024 ** 1, "MB": 1024 * 2, "GB": 1024 ** 3, "byte": 1}
    divide_by = unit_map[unit]
    optimize_dtypes = []

    for col in df.columns:
        as_type = get_smallest_valid_conversion(df[col], optimize)
        nbytes = df[col].memory_usage(deep=True)
        proposed_memory, proposed_dtype, ram_usage_improvement = None, None, None
        if as_type:
            (
                proposed_bytes,
                proposed_dtype,
                ram_usage_improvement,
            ) = get_improvement(as_type, nbytes)
            # If improvement is found, replace the attributes
            proposed_memory = proposed_bytes / divide_by if proposed_bytes else None
            ram_usage_improvement = (
                ram_usage_improvement / divide_by if ram_usage_improvement else None
            )
            proposed_dtype = proposed_dtype
        row = Row(
            column=col,
            current_dtype=df[col].dtype,
            current_memory=nbytes / divide_by,
            proposed_memory=proposed_memory,
            proposed_dtype=proposed_dtype,
            ram_usage_improvement=ram_usage_improvement,
        )
        optimize_dtypes.append(row)
    columns = [
        "Column",
        "Current dtype",
        "Proposed dtype",
        f"Current Memory ({unit})",
        f"Proposed Memory ({unit})",
        f"Ram Usage Improvement ({unit})",
    ]
    report_df = pd.DataFrame(optimize_dtypes, columns=columns)
    report_df["Ram Usage Improvement (%)"] = (
        report_df[f"Ram Usage Improvement ({unit})"]
        / report_df[f"Current Memory ({unit})"]
        * 100
    )
    report_df = report_df.set_index("Column")
    return report_df

In [ ]:
#| slow
# sell_prices.csv.zip 
# Source data: https://www.kaggle.com/c/m5-forecasting-uncertainty/

df = pd.read_csv('data/sell_prices.csv')

In [ ]:
#| slow
report_on_dataframe(df)

,Current dtype,Proposed dtype,Current Memory (MB),Proposed Memory (MB),Ram Usage Improvement (MB),Ram Usage Improvement (%)
Column,,,,,,
store_id,object,category,203763.920410,3340.907715,200423.012695,98.360403
item_id,object,category,233039.977539,6824.677734,226215.299805,97.071456
wm_yr_wk,int64,int16,26723.191406,6680.844727,20042.346680,74.999825
sell_price,float64,None,26723.191406,NaN,NaN,NaN


`report_on_dataframe` shows you the possible dtype conversion and the improvement. Note that the library try to optimize the memory base on __current__ values of the data, you should still be careful about overflow for further transformation. 

In [ ]:
#| export
def optimize_dtypes(df: pd.DataFrame, proposed_df: pd.DataFrame) -> pd.DataFrame:
    new_df = df.copy()
    for col in df.columns:
        new_dtype = proposed_df.loc[col, "Proposed dtype"]
        if new_dtype:
            new_df[col] = new_df[col].astype(new_dtype)
    return new_df

In [ ]:
#| include: false
def test_ser_ints():
    # check for low simple int
    ser = pd.Series([1] * 3)
    as_type = count_errors(ser, "int32")
    assert as_type.nbr_different == 0
    as_type = count_errors(ser, "int16")
    assert as_type.nbr_different == 0
    as_type = count_errors(ser, "int8")
    assert as_type.nbr_different == 0

    # check for int needing bigger than int16
    ser = pd.Series([65536] * 3)
    as_type = count_errors(ser, "int32")
    assert as_type.nbr_different == 0
    as_type = count_errors(ser, "int16")
    assert as_type.nbr_different == 3
    as_type = count_errors(ser, "int8")
    assert as_type.nbr_different == 3

In [ ]:
if __name__ == "__main__":
    print("Given a dataframe, check for lowest possible conversions:")

    nbr_rows = 100
    df = pd.DataFrame()
    df["a"] = [0] * nbr_rows
    df["b"] = [256] * nbr_rows
    df["c"] = [65_536] * nbr_rows
    df["d"] = [1_100.0] * nbr_rows
    df["e"] = [100_101.0] * nbr_rows
    df["str_a"] = ["hello"] * nbr_rows
    df["str_b"] = [str(n) for n in range(nbr_rows)]
    report_on_dataframe(df)

    print("convert_dtypes does a slightly different job:")
    print(df.convert_dtypes())

Given a dataframe, check for lowest possible conversions:
convert_dtypes does a slightly different job:
    a    b      c     d       e  str_a str_b
0   0  256  65536  1100  100101  hello     0
1   0  256  65536  1100  100101  hello     1
2   0  256  65536  1100  100101  hello     2
3   0  256  65536  1100  100101  hello     3
4   0  256  65536  1100  100101  hello     4
.. ..  ...    ...   ...     ...    ...   ...
95  0  256  65536  1100  100101  hello    95
96  0  256  65536  1100  100101  hello    96
97  0  256  65536  1100  100101  hello    97
98  0  256  65536  1100  100101  hello    98
99  0  256  65536  1100  100101  hello    99

[100 rows x 7 columns]
